In [4]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [49]:
!pip install stable-baselines3[extra]
!pip install gym

In [3]:
import time
import gym 
import pygame
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [6]:
#pygame.init()

environment_name = "CartPole-v0"
env = gym.make(environment_name, render_mode='human')

c:\Users\abdul\anaconda3.1\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [52]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 

    while not done:
        env.render()
        time.sleep(0.05)  # Add a delay to see the render
        action = env.action_space.sample()
        results = env.step(action)
        n_state, reward, done, info = results[:4]  # Unpack only the first 4 values
        score += reward
    print(f'Episode:{episode} Score:{score}')

env.close()
pygame.quit()

c:\Users\abdul\anaconda3.1\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:16.0
Episode:2 Score:20.0
Episode:3 Score:10.0
Episode:4 Score:18.0
Episode:5 Score:20.0


In [68]:
env.action_space.sample()

1

# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [53]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [54]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.5239766e+00,  1.7930514e+38, -2.8381222e-01, -7.9300824e+37],
      dtype=float32)

# 3. Train an RL Model

In [6]:
log_path= os.path.join("Training","Logs")

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path)

c:\Users\abdul\anaconda3.1\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [4]:
model.learn(total_timesteps=20000)

NameError: name 'model' is not defined

# 4. Save and Reload Model

In [7]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_Cartpole')

In [10]:
model.save(PPO_path)

In [11]:
del model

In [12]:
model = PPO.load(PPO_path, env=env)

# 5. Evaluation

In [13]:
from stable_baselines3.common.evaluation import evaluate_policy

In [14]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

c:\Users\abdul\anaconda3.1\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\abdul\anaconda3.1\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(200.0, 0.0)

In [15]:
env.close()

# 6. Test Model

In [19]:
pygame.init()

environment_name = "CartPole-v0"
env = gym.make(environment_name, render_mode='human')

In [17]:
# without the model

episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 

    while not done:
        env.render()
        time.sleep(0.05)  # Add a delay to see the render
        action = env.action_space.sample()
        results = env.step(action)
        n_state, reward, done, info = results[:4]  # Unpack only the first 4 values
        score += reward
    print(f'Episode:{episode} Score:{score}')

env.close()
pygame.quit()

Episode:1 Score:15.0
Episode:2 Score:28.0
Episode:3 Score:20.0
Episode:4 Score:24.0
Episode:5 Score:20.0


In [20]:
# with model
episodes = 5

for episode in range(1, episodes + 1):
    obs, _ = env.reset()  # Extract obs from the tuple
    done = False
    truncated = False
    score = 0

    while not (done or truncated):
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)  # Handle truncated value
        score += reward
    print(f'Episode: {episode} Score: {score}')

env.close()
pygame.quit()

Episode: 1 Score: 200.0
Episode: 2 Score: 200.0
Episode: 3 Score: 200.0
Episode: 4 Score: 200.0
Episode: 5 Score: 200.0


In [21]:
env.close()

# 6. Viewing Logs in Tensorboard

In [8]:
!pip install tensorboard

In [13]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [14]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
!tensorboard --logdir={training_log_path}

^C


# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()